Bnb interval method with convexity

In [1]:
from sympy import *
import interval as ival
import intervaldrv as ivald
import matplotlib.pyplot as plt
import numpy as np
import math
from problems import *
import bnbcore as bc
import intcore as ic

In [2]:
ivald_dict = {"cos" : ivald.cos, "sin" : ivald.sin, "exp" : ivald.exp, "log" : ivald.log}

In [3]:
solve_options = {'use_1st_form' : True}

In [4]:
def split_iconv(s):
    a = s.x[0]
    b = s.x[1]
    c = s.mid()
    sl = ivald.IntervalDrv([a, c])
    sr = ivald.IntervalDrv([c, b])
    return [sl, sr]

In [5]:
def solve(problems,key):
    problem = problems[key]
    main_int = ivald.IntervalDrv(problem['ab'])
    f = lambdify(x, problem['f'])
    fi = lambdify(x, problem['f'], ivald_dict)
    def fic(x): 
        rvfi = fi(x)
        c = ic.mid(x)
        fc = f(c)
        xn = ic.first_form(x, rvfi.dx, [c, c], [fc, fc])
        xnn = ic.cap(rvfi.x, xn)
        return xnn
    if solve_options['use_1st_form']:
        rv = bc.bnb(problem, [main_int], 1e5, f, fic, split_iconv)
    else:
        rv = bc.bnb(problem, [main_int], 1e5, f, fi, split_iconv)
    return rv

In [6]:
def vizual(flist, a, b):
    colors = ['r-', 'b-', 'g-']
    step = (b-a)/100.
    ta = np.arange(a, b + step, step)
    ln = len(ta)
    fta = np.arange(ln).astype(float)
    j = 0
    for f in flist:
        for i in range(ln):
            fta[i] = f(ta[i])
        plt.plot(ta, fta, colors[j])
        j = j + 1
    plt.show()

In [9]:
def run_tests(tests, domon, doconv, taylor):
    ivald.options['domon'] = domon
    ivald.options['doconv'] = doconv
    solve_options['use_1st_form'] = taylor
    steps = []
    for k in tests.keys():
        s = solve(tests, k)
        steps += [s['steps']]
    return steps

In [12]:
steps = []
steps += [run_tests(diff_problems, False, False, False)]
steps += [run_tests(diff_problems, True, True, False)]
steps += [run_tests(diff_problems, False, False, True)]
steps += [run_tests(diff_problems, True, True, True)]
print(steps)

[[35, 72953, 63, 159, 183, 11, 273, 65, 189, 857, 51], [15, 255, 21, 17, 39, 11, 31, 25, 49, 31, 19], [29, 311, 37, 31, 69, 11, 49, 37, 91, 75, 27], [15, 101, 21, 17, 39, 11, 31, 25, 49, 31, 19]]
